<p style="font-size:15pt; text-align:center">
    Introduction to Data Science
</p>
<p style="font-size:20pt; text-align:center">
    Working with network data
</p>

In [ ]:
from collections import Counter # dict subclass for counting hashable objects
from pprint import pprint  # pretty printer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
counting_1 = [1,2,3,3,4,5,6,6,7,7,7]

In [ ]:
# 计数方法1
pd.Series(counting_1).value_counts()

In [ ]:
# 计数方法2
np.unique(counting_1,return_counts=True)

In [ ]:
# 计数方法3
Counter([1,2,3,3,4,5,6,6,7,7,7]) # 计数器

In [ ]:
# 计数方法1
pd.Series(list('aacdsceqwacwdsaceaew')).value_counts()

In [ ]:
# 计数方法2
np.unique(list('aacdsceqwacwdsaceaew'),return_counts=True)

In [ ]:
# 计数方法3
Counter('aacdsceqwacwdsaceaew')

# Edge List

Let us start by defining a list of edges. This will give us our first "dataset" to work with. The network structure is:

<img src="file/graph_diagram.png" width=400>


In [ ]:
edge_list = [
    ('A', 'B'),
    ('A', 'C'),
    ('A', 'E'),
    ('B', 'C'),
    ('C', 'D'),
    ('C', 'E'),
    ('D', 'E')]

This is a particularly useful representation as many datasets are distributed in this (or a closely related) format. From this list, we can easily measure the number of edges that constitute our network. It's main limitations are that it has no way to explictly take into account disconnected nodes (it only accounts for nodes that are part of edges) and no indication on whether it is directed or not.

这是一种特别有用的表示，因为许多数据集都以这种(或密切相关的)格式分布。从这个列表中，我们可以很容易地测量构成我们网络的边的数量。

它的主要限制是，它没有办法明确地考虑断开的节点(它只考虑了边的一部分)，并且没有指示它是否是有向的。

In [29]:
number_edges = len(edge_list)
print(number_edges)

7


To get the number of node is a bit trickier. We must go edge by edge and keep track of all new nodes. For efficiency, we use a set to automatically remove duplicates

In [39]:
nodes = set()

for edge in edge_list:
    nodes.update(edge) # Update a set with the union of itself and others.
    
number_nodes = len(nodes)
print(number_nodes)

5


Now we know that we have 5 nodes and 7 edges in our network. The node ids are:

In [40]:
nodes

{'A', 'B', 'C', 'D', 'E'}

# Adjacency List

A closely related data structure to the edge list is the adjacency list. In this formulation, we use a dictionary to map each node to its set of neighbors

In [ ]:
adj_list = {}

for node_i, node_j in edge_list:
    if node_i not in adj_list:
        adj_list[node_i] = set()

    adj_list[node_i].add(node_j)

Our adjaceny list is then:

In [ ]:
pprint(adj_list)

In this approach we inherently assumed that our network is directed (or, equivalently, that both edge directions are present in the data). To generate an undirected version we must make a simple modification to our code to manually add the opposite direction edge

在这种方法中，我们固有地假设我们的网络是有向的(或者，等价地，两个边缘方向都存在于数据中)。要生成无方向的版本，我们必须对代码做一个简单的修改，手动添加相反方向的边

In [ ]:
adj_list = {}

for node_i, node_j in edge_list:
    if node_i not in adj_list:
        adj_list[node_i] = set() # 'set' is used to prevent accidental multiple edges
    
    adj_list[node_i].add(node_j)
    
    # Manually add the opposite direction edge
    if node_j not in adj_list:
        adj_list[node_j] = set()
    
    adj_list[node_j].add(node_i)

The undirected adjacency list represenation is then:

In [41]:
print(adj_list)

{'A': {'B', 'E', 'C'}, 'B': {'C', 'A'}, 'C': {'D', 'B', 'E', 'A'}, 'E': {'D', 'C', 'A'}, 'D': {'E', 'C'}}


# Adjacency Matrix

We now move on to generating an Adjacency Matrix view of the network. For this we must have two things: 

- the number of nodes in the network
- A mapping between the original node ids and a sequential numerical ID

We start by building out the numerical ID mapping. As we do, we get the number of nodes for free

In [42]:
node_id = {}
node_count = 0

for node_i, node_j in edge_list:
    if node_i not in node_id:
        node_id[node_i] = node_count
        node_count += 1
    
    # Make sure we have an id for both nodes
    # This is necessary, irregardless of whether the network is directed or undirected
    if node_j not in node_id:
        node_id[node_j] = node_count
        node_count += 1

We can check that each of the original node ids is correctly mapped to a sequential number

In [43]:
node_id

{'A': 0, 'B': 1, 'C': 2, 'E': 3, 'D': 4}

Finally, we are ready to build our adjacency matrix. We start by declaring the data structurewe will use

In [44]:
adj_matrix = np.zeros((node_count, node_count), dtype='int')

And we can now populate the matrix entries. For generality, we'll include a flag to control wether or not the graph is directed. As we don't have any weights associated with our edges, we assign to each of them a value of 1.

In [45]:
is_directed = False

for node_i, node_j in edge_list:    
    # Get the correct node ids
    node_i = node_id[node_i]
    node_j = node_id[node_j]
    
    adj_matrix[node_i, node_j] = 1 # Unweighted network

    if not is_directed:
        adj_matrix[node_j, node_i] = 1 # Undirected networks

Our Adjacency Matrix is then:

In [46]:
adj_matrix

array([[0, 1, 1, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0]])

As we can see, the Adjacency matrix representation is very wasteful, using 25 values to store a 7 (14) edge network plus 5 dictionary entries for the id mappings.

# Adjacency Dict

The final graph representation we will explore is the Adjacency Dict. This is a generalization of the Adjacency List covered above that is a bit more flexible and is able to accuratly account for diconnected nodes, weights, etc. For this we will need two datastructures, one to store node information and one for edges. For the sake of flexbility, we will use dicts for both.

In [47]:
nodes = {}
edges = {}
is_directed = False

And we can now populate them from our original edge_list

In [48]:
for node_i, node_j in edge_list:
    if node_i not in nodes:
        nodes[node_i] = {}
        edges[node_i] = {}
        
    if node_j not in nodes:
        nodes[node_j] = {}
        
        if not is_directed:
            edges[node_j] = {}
    
    edges[node_i][node_j] = {}
    
    if not is_directed:
        edges[node_j][node_i] = {}

Our set of nodes is:

In [49]:
nodes

{'A': {}, 'B': {}, 'C': {}, 'E': {}, 'D': {}}

Where we chose to use dictinaries to allow for the storage of node attributes. Further, our edges are now:

In [50]:
edges

{'A': {'B': {}, 'C': {}, 'E': {}},
 'B': {'A': {}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}},
 'D': {'C': {}, 'E': {}}}

Where we once again opted to associate a dictionary to each edge. This gives us the ability to associate edge information (such as weights, etc) to each node.

# The End

**Source**

This notebook was adapted from:
* Data 8: The Foundations of Data Science
* Introduction to Data Science by Michael Szell
* Introduction to Data Science and Visualization by James Bagrow
* Bruno Gonçalves, Data4Sci